In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version: 2.4.1


In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# normalize to [0,1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension

11493376/11490434 [==============================] - 2s 0us/step


In [9]:
x_train.shape

(60000, 28, 28)

In [10]:
x_test.shape

(10000, 28, 28)

In [11]:
x_train = x_train[...,tf.newaxis].astype("float32")

In [12]:
x_train.shape

(60000, 28, 28, 1)

In [13]:
type(x_train)

numpy.ndarray

In [14]:
x_test = x_test[..., tf.newaxis].astype("float32")

In [15]:
train_ds = tf.data.Dataset.from_tensor_slices(
 (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

AttributeError: 'BatchDataset' object has no attribute 'shape'

In [17]:
train_ds

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.uint8)>

In [18]:
class MyModel(Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.conv1 = Conv2D(32,3,activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
model = MyModel()

In [19]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [20]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [22]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=Tru is oly needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout)
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [24]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different 
    # behavior during training versus inference (e.g. Dropouts).
    predictions = model( images, training=False)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [25]:
EPOCHS = 5
for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
        
    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.14060652256011963, Accuracy: 95.71499633789062, Test Loss: 0.05814700946211815, Test Accuracy: 98.0199966430664
Epoch 2, Loss: 0.04356954246759415, Accuracy: 98.58333587646484, Test Loss: 0.050071876496076584, Test Accuracy: 98.3499984741211
Epoch 3, Loss: 0.0227091982960701, Accuracy: 99.2699966430664, Test Loss: 0.05306507274508476, Test Accuracy: 98.43999481201172
Epoch 4, Loss: 0.013430717401206493, Accuracy: 99.54833221435547, Test Loss: 0.06309595704078674, Test Accuracy: 98.44999694824219
Epoch 5, Loss: 0.009729649871587753, Accuracy: 99.6683349609375, Test Loss: 0.062089141458272934, Test Accuracy: 98.43999481201172
